In [160]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import json
import difflib

In [161]:
# Get NBA Team IDs
teams = pd.DataFrame(teams.get_teams())

In [168]:
year = 2018


# Get Past Team Schedules
def getRegularSeasonSchedule(seasonYear,teamID):
    season = str(seasonYear) + "-" + str(seasonYear+1)[-2:]
    print(season)
    teamGames = cumestatsteamgames.CumeStatsTeamGames(league_id = '00',season = season ,
                                                                  season_type_all_star='Regular Season',
                                                                  team_id = teamID).get_normalized_json()

    return pd.DataFrame(json.loads(teamGames)['CumeStatsTeamGames'])

teamGames = getRegularSeasonSchedule(year,1610612756)
teamGames

2018-19


,MATCHUP,GAME_ID
0,04/09/2019 Suns at Mavericks,0021801216
1,04/07/2019 Suns at Rockets,0021801200
2,04/05/2019 Pelicans at Suns,0021801189
3,04/03/2019 Jazz at Suns,0021801174
4,04/01/2019 Cavaliers at Suns,0021801160
...,...,...
77,10/27/2018 Suns at Grizzlies,0021800080
78,10/24/2018 Lakers at Suns,0021800060
79,10/22/2018 Suns at Warriors,0021800047
80,10/20/2018 Suns at Nuggets,0021800033


In [169]:
# Format Data Frame Columns and Get Home and Away Team IDs

def getGameDate(matchup):
    return matchup.partition(' at')[0][:10]

def getHomeTeam(matchup):
    return matchup.partition(' at')[2]

def getAwayTeam(matchup):
    return matchup.partition(' at')[0][10:]

def getTeamIDFromNickname(nickname):
    return teams.loc[teams['nickname'] == difflib.get_close_matches(nickname,teams['nickname'],1)[0]].values[0][0] 

teamGames['GAME_DATE'] = pd.to_datetime(teamGames['MATCHUP'].map(getGameDate))
teamGames['HOME_TEAM_NICKNAME'] = teamGames['MATCHUP'].map(getHomeTeam)
teamGames['HOME_TEAM_ID'] = teamGames['HOME_TEAM_NICKNAME'].map(getTeamIDFromNickname)
teamGames['AWAY_TEAM_NICKNAME'] = teamGames['MATCHUP'].map(getAwayTeam)
teamGames['AWAY_TEAM_ID'] = teamGames['AWAY_TEAM_NICKNAME'].map(getTeamIDFromNickname)

In [170]:
#teamGames['MATCHUP'][0].partition('at')[0][10:]
teamGames

,MATCHUP,GAME_ID,GAME_DATE,HOME_TEAM_NICKNAME,HOME_TEAM_ID,AWAY_TEAM_NICKNAME,AWAY_TEAM_ID
0,04/09/2019 Suns at Mavericks,0021801216,2019-04-09,Mavericks,1610612742,Suns,1610612756
1,04/07/2019 Suns at Rockets,0021801200,2019-04-07,Rockets,1610612745,Suns,1610612756
2,04/05/2019 Pelicans at Suns,0021801189,2019-04-05,Suns,1610612756,Pelicans,1610612740
3,04/03/2019 Jazz at Suns,0021801174,2019-04-03,Suns,1610612756,Jazz,1610612762
4,04/01/2019 Cavaliers at Suns,0021801160,2019-04-01,Suns,1610612756,Cavaliers,1610612739
...,...,...,...,...,...,...,...
77,10/27/2018 Suns at Grizzlies,0021800080,2018-10-27,Grizzlies,1610612763,Suns,1610612756
78,10/24/2018 Lakers at Suns,0021800060,2018-10-24,Suns,1610612756,Lakers,1610612747
79,10/22/2018 Suns at Warriors,0021800047,2018-10-22,Warriors,1610612744,Suns,1610612756
80,10/20/2018 Suns at Nuggets,0021800033,2018-10-20,Nuggets,1610612743,Suns,1610612756


In [172]:

seasonYear = 2018
homeTeamID = 1610612743
gameID = '0021800033'
# Get Game Data for Home Team


season = str(seasonYear) + "-" + str(seasonYear+1)[-2:]
gameStats = cumestatsteam.CumeStatsTeam(game_ids=gameID,league_id ="00",
                                           season=season,season_type_all_star="Regular Season",
                                           team_id = homeTeamID).get_normalized_json()

gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])
gameStats

#IF I KNOW THE AWAY TEAM THEN I KNOW THE OPPONENT AND ONLY HAVE TO CALL A GAME ONCE

,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,TOT_REB,AST,PF,STL,TOTAL_TURNOVERS,BLK,PTS,AVG_REB,AVG_PTS,DQ
0,Denver,Nuggets,1610612743,1,0,1,0,0,0,0,...,61,28,31,12,18,6,119,61.0,119.0,1
1,OPPONENTS,,0,0,1,0,0,0,1,0,...,47,20,26,6,19,4,91,47.0,91.0,0
